# Membership Calculations

Trying to understand what percentage of PCA members are in the churches with FFO...

Jonathan Barlow (@barlowjon)


## First, Load the Data

In [66]:
# Prepare the libraries
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Load the two CSV files

# this is the 'save the pca' dataset
# https://www.savethepca.com/downloads/
left = pd.read_csv("stats_left.csv") 

# this is the datafile from the PCA sans headers and footers
# https://presbyteryportal.pcanet.org/Report/StatsReport
right = pd.read_csv("stats_right.csv")

left.describe()

,Presbytery,Church,Phone,Website,Email,Pastor,Review Date,Elders on Website? (Y/N),Deacons on Website? (Y/N),Other Staff on Website? (Y/N),Shepherdesses? (Y/N),Deaconesses? (Y/N),Website Status
count,1964,1964,1963,1964,1964,1964,1528,1528,1526,1528,1185,904,1964
unique,87,1365,1830,1734,1777,1715,12,2,2,2,2,2,8
top,Mississippi Valley,First Presbyterian Church,-,-,-,-,09-27-25,Y,Y,Y,N,N,Functional
freq,50,54,117,229,180,235,672,1159,837,968,1151,813,1528


## Pull Out Just the Churches with FFO

In [67]:
leftFfo = left[(left["Shepherdesses? (Y/N)"].astype(str).str.upper().str.contains("Y")) |
              (left["Deaconesses? (Y/N)"].astype(str).str.upper().str.contains("Y"))].copy()

leftFfo.describe()

,Presbytery,Church,Phone,Website,Email,Pastor,Review Date,Elders on Website? (Y/N),Deacons on Website? (Y/N),Other Staff on Website? (Y/N),Shepherdesses? (Y/N),Deaconesses? (Y/N),Website Status
count,111,111,111,111,111,111,111,111,111,111,110,107,111
unique,43,100,103,111,109,106,9,2,2,2,2,2,1
top,Metropolitan New York,Redeemer Presbyterian Church,-,ascensionphx.org,-,-,09-27-25,Y,Y,Y,N,Y,Functional
freq,9,6,9,1,3,6,40,109,99,110,76,91,111


Okay, looks like we have 111 churches with FFO.

## Fix Missing Pastor Names

Since we're going to merge on pastor and church name, we need to be able to match like to like in the two datasets. The FFO file had a dash for missing pastors.

In [68]:
right['Pastor'] = right['Pastor'].fillna('-').astype(str).str.strip()
right.loc[right['Pastor'] == '', 'Pastor'] = '-'

# Set Calvary Church's Pastor to Match FFO Data
right.loc[
    (right['City'] == 'Willow Grove') &
    (right['St'] == 'PA') &
    (right['Church'] == 'Calvary Presbyterian Church'),
    'Pastor'
] = 'Rev. Angel Gomez'


# Redeemer (has three pastors listed in the pca stats sheet)

right.loc[
    (right['City'] == 'Winston-Salem') &
    (right['St'] == 'NC') &
    (right['Church'] == 'Redeemer Presbyterian Church'),
    'Pastor'
] = 'Rev. Giorgio W.K. Hiatt'


# King's Cross Flushing - FFO data had the wrong pastor

leftFfo.loc[
    (leftFfo['Presbytery'] == 'Metropolitan New York') &
    (leftFfo['Church'] == "King's Cross Church"),
    'Pastor'
] = 'James Park'

# Redeemer San Antonio

right.loc[
    (right['City'] == 'San Antonio') &
    (right['St'] == 'TX') &
    (right['Church'] == 'Redeemer Presbyterian Church'),
    'Pastor'
] = 'Rev. James Paul Hahn Jr.'


# Christ the King Dorchester has a data quality problem (extra space in name) in the pca stats data

right.loc[
    (right['City'] == 'Dorchester') &
    (right['St'] == 'MA') &
    (right['Church'] == 'Christ the King  - Dorchester'),
    'Church'
] = 'Christ the King - Dorchester'


## Merge the PCA Stats into the Data

In [69]:
# Normalize the Church name and Pastor columns for better matching
def normalize_church(s):
    return str(s).strip().lower()

def normalize_pastor(s):
    s = str(s).strip().lower()
    parts = s.split()
    return parts[-1] if parts else ''

leftFfo['Church_norm'] = leftFfo['Church'].apply(normalize_church)
leftFfo['Pastor_norm'] = leftFfo['Pastor'].apply(normalize_pastor)
right['Church_norm'] = right['Church'].apply(normalize_church)
right['Pastor_norm'] = right['Pastor'].apply(normalize_pastor)

# Merge the right sheet into the left sheet, keeping all rows from left
merged = pd.merge(leftFfo, right, on=['Church_norm', 'Pastor_norm'], how='left', suffixes=('_left', '_right'))

merged

,Presbytery,Church_left,Phone,Website,Email,Pastor_left,Review Date,Elders on Website? (Y/N),Deacons on Website? (Y/N),Other Staff on Website? (Y/N),Shepherdesses? (Y/N),Deaconesses? (Y/N),Website Status,Church_norm,Pastor_norm,City,St,Church_right,Pastor_right,Stat Year,Comm,Non Comm,Total,Prof Child,Prof Adult,Trans Letter,Reaffir-mation,Re-stored,Total.1,Death,Trans,Rem'v fr Roll,Discipl,Total.2,Unnamed: 19,Adult Bapt,Infant Bapt,Ruling Elders,Deacons,Family Units,Sunday School Attend,Small Group Attend,Est Morn Attend,Grades *Included,Total Enrollment,Tithes & Offerings,Special\nCauses,Building Fund Offering,Other Contrib,Total Contrib,Other Income,Total Church Income,Unnamed: 37,Admini-stration,Disciple-ship Min.,Mission to N. America,Mission to the World,Covenant College,Covenant Seminary,Ridge\nHaven,Reformed University Ministries,PCA Office Building,Other,Total.3,Presbytery Operation,Christian Education,Home Missions,Internat'l\nMissions,Other Pres-bytery Min,Total.4,Unnamed: 55,Local Ministries,Mercy Ministires,PCA Internat'l\nMissions,PCA Home Mission,Non-PCA Mission,PCA Schools,Non-PCA Schools,Total.5,TOTAL Benevolent Disbursements,Current Expenses,Building Fund,GRAND TOTAL: All Disbursements,Benevol/\nGrand Total,Per Capita Giving
0,Arizona,Ascension Church of Phoenix,601-209-5609,ascensionphx.org,gray@ascensionphx.org,Rev. Gray A. Ewing,09-24-25,Y,Y,Y,N,Y,Functional,ascension church of phoenix,ewing,Phoenix,AZ,Ascension Church of Phoenix,Gray A. Ewing,2022.0,100,19,119,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$362,506",NaN,NaN,NaN,"$362,506",NaN,"$362,506",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,$0,NaN,NaN,NaN,NaN,NaN,$0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,$0,$0,"$385,400",NaN,"$385,400",0%,"$3,625"
1,Arizona,New Valley Church,480-940-5560,www.newvalleychurch.org,amanda@newvalleychurch.org,Rev. Scott Brown,09-24-25,Y,Y,Y,N,Y,Functional,new valley church,brown,Chandler,AZ,New Valley Church,Scott Brown,2024.0,324,101,425,3.0,0.0,0.0,36.0,0.0,39.0,0.0,6.0,0.0,2.0,8.0,NaN,4.0,7.0,5.0,4.0,123,100.0,170,425,NaN,NaN,"$1,331,527","$59,773",NaN,NaN,"$1,391,300","$37,713","$1,429,013",NaN,"$3,783",NaN,NaN,NaN,"$3,250",NaN,NaN,"$2,200",NaN,NaN,"$9,233",NaN,NaN,NaN,NaN,NaN,"$21,335",NaN,"$3,000","$48,324","$3,652","$2,400","$34,373",NaN,NaN,"$91,749","$122,317","$1,161,678",NaN,"$1,283,995",10%,"$4,294"
2,Blue Ridge,Hope Presbyterian Church of Crozet,434-829-2181,www.hopecrozet.org,office@hopecrozet.org,Rev. Todd Johnson,09-25-25,Y,N,Y,Y,NaN,Functional,hope presbyterian church of crozet,johnson,Crozet,VA,Hope Presbyterian Church of Crozet,Todd Johnson,2024.0,50,9,59,1.0,NaN,2.0,NaN,NaN,3.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,4.0,NaN,30,NaN,30,60,NaN,NaN,"$197,889","$5,645",NaN,NaN,"$203,534",NaN,"$203,534",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"$5,000","$5,000","$2,223",NaN,NaN,NaN,NaN,"$2,223",NaN,"$3,200","$9,264","$1,800","$1,500","$11,880",NaN,NaN,"$27,644","$34,867","$183,474",NaN,"$218,341",16%,"$4,071"
3,Blue Ridge,Trinity Presbyterian Church,434-977-3700,www.trinitycville.org,trinity@trinitycville.org,Rev. Christopher Paul Colquitt,09-23-25,Y,Y,Y,Y,N,Functional,trinity presbyterian church,colquitt,Charlottesville,VA,Trinity Presbyterian Church,Christopher Paul Colquitt,2024.0,"1,104",214,"1,318",30.0,2.0,25.0,69.0,0.0,126.0,0.0,24.0,0.0,0.0,24.0,NaN,11.0,19.0,16.0,15.0,385,60.0,298,634,P,37.0,"$3,030,661","$164,312",NaN,NaN,"$3,194,973","$310,915","$3,505,888",NaN,"$5,028","$1,056","$3,924","$4,529","$1,434","$1,509",$604,"$2,717",NaN,NaN,"$20,801",NaN,NaN,NaN,NaN,NaN,"$38,000",NaN,"$140,388","$269,807","$22,950","$25,000","$178,887",NaN,NaN,"$637,032","$695,833","$2,216,232","$126,667","$3,038,732",23%,"$2,894"
4,Calvary,Downtown Presbyterian Church,864-608-5529,www.downtownpres.org,updates@downtownpres.org,Rev. Brian C. Habig,09-25-25,Y,Y,Y,N,Y,Functional,downtown presbyterian church,habig,Greenville,SC,Downtown Presbyterian Church,Brian C. Habig,2024.0,"1,021",300,"1,32

# Calculate the Percentages of Total Churchgoers Under FFO Churches

In [78]:

merged['Total'] = pd.to_numeric(merged['Total'].astype(str).str.replace(',', '').str.strip(), errors='coerce')
right['Total'] = pd.to_numeric(right['Total'].astype(str).str.replace(',', '').str.strip(), errors='coerce')

merged['Comm'] = pd.to_numeric(merged['Comm'].astype(str).str.replace(',', '').str.strip(), errors='coerce')
right['Comm'] = pd.to_numeric(right['Comm'].astype(str).str.replace(',', '').str.strip(), errors='coerce')


# calc for churchgoers (total of comm, non-comm)
totalUnderFfo = merged['Total'].sum()
totalGoers = right['Total'].sum()

percentUnderFfo = (totalUnderFfo / totalGoers)*100

print(totalUnderFfo," attendees, representing at least ",percentUnderFfo,"% of PCA Attendees are under FFO")

45070.0  attendees, representing at least  11.41021324212589 % of PCA Attendees are under FFO


In [80]:
# total for communing

totalCommUnderFfo = merged['Comm'].sum()
totalComm = right['Comm'].sum()

percentCommUnderFfo = (totalCommUnderFfo / totalComm)*100

print(totalCommUnderFfo," communing members, representing at least ",percentCommUnderFfo,"% of communing PCA Members are under FFO")

34998.0  communing members, representing at least  11.191982245303896 % of communing PCA Members are under FFO
